###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from functools import reduce
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../../Functions and Dictionaries/") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

In [2]:
#functions
def percent(x, y):
        try:
            return ((x/y)*100)
        except ZeroDivisionError:
            return 0
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y

# Age Well  

The following code outlines data collection, formatting, cleaning, and calculations for a data request for Age Well Tennessee.  

The following data is collected:  
+ total population 65+  
+ income  
+ caregiver for grandkids  
+ own vs rent  
+ cost burden  
+ living alone  
+ educational attainment  
+ poverty  

Most of this was directly requested, but some was added on by research staff as the request asked for other compelling data to make a case for the need for affordable housing for seniors.

In [3]:
conn = sq.connect('../../../Data-Pipelines/Outputs/CensusBureau.db')
#PL2020
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_PL_2020]', conn)
PL2020 = pd.DataFrame(sql_query)
PL2020.drop(columns = 'Source', inplace = True)
#ACS52021
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS1_2021_1]', conn)
ACS52021_1 = pd.DataFrame(sql_query)
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS1_2021_2]', conn)
ACS52021_2 = pd.DataFrame(sql_query)
ACS52021 = ACS52021_1.merge(ACS52021_2, on = ['NAME', 'GEO_ID', 'Source'])
ACS52021.drop(columns = 'Source', inplace = True)
dfs2021 = [ACS52021, PL2020]
twothousandtwentyone = reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID'], how='outer'), dfs2021)

In [4]:
data = twothousandtwentyone.loc[(twothousandtwentyone['NAME'] == 'Tennessee')]

In [5]:
cols = ['NAME', 'Age:65 and Older', 'Age%:65 and Older',

        #med hhincome age group
        'HHIncome:Median for Householder Under 65 Over', 
        #med hhincome
        'Median Household Income',
        #household income by age group
        'HHIncome:65 Over Less than 10,000', 'HHIncome%:65 Over Less than 10,000', 
        'HHIncome:65 Over 10 to 14,999', 'HHIncome%:65 Over 10 to 14,999', 'HHIncome:65 Over 15 to 19,999', 'HHIncome%:65 Over 15 to 19,999', 
        'HHIncome:65 Over 20 to 24,999', 'HHIncome%:65 Over 20 to 24,999', 'HHIncome:65 Over 25 to 29,999', 'HHIncome%:65 Over 25 to 29,999', 
        'HHIncome:65 Over 30 to 34,999', 'HHIncome%:65 Over 30 to 34,999', 'HHIncome:65 Over 35 to 39,999', 'HHIncome%:65 Over 35 to 39,999', 
        'HHIncome:65 Over 40 to 44,999', 'HHIncome%:65 Over 40 to 44,999', 'HHIncome:65 Over 45 to 49,999', 'HHIncome%:65 Over 45 to 49,999', 
        'HHIncome:65 Over 50 to 59,999', 'HHIncome%:65 Over 50 to 59,999', 'HHIncome:65 Over 60 to 74,999', 'HHIncome%:65 Over 60 to 74,999', 
        'HHIncome:65 Over 75 to 99,999', 'HHIncome%:65 Over 75 to 99,999', 'HHIncome:65 Over 100 to 124,999', 'HHIncome%:65 Over 100 to 124,999', 
        'HHIncome:65 Over 125 to 149,999', 'HHIncome%:65 Over 125 to 149,999', 'HHIncome:65 Over 150 to 199,999', 'HHIncome%:65 Over 150 to 199,999', 
        'HHIncome:65 Over 200K or More', 'HHIncome%:65 Over 200K or More',
        #cost burden for age group
        'CBAge:Cost Burdened Households Householder 65 Over', 'CBAge%:Households Cost Burdened Householder 65 Over',
        #tenure by age group
        'Homeownership:Householders Age 60 and Over', 'Homeownership:Homeowners Age 60 and Over', 'Homeownership%:Homeowners Age 60 and Over', 
        'Homeownership%:Age 60 and Over Householders Homeowners', 'Homeownership:Renters Age 60 and Over', 'Homeownership%:Renters Age 60 and Over', 
        'Homeownership%:Age 60 and Over Householders Renters', 'Homeownership:Householders Age 65 and Over', 'Homeownership:Homeowners Age 65 and Over', 
        'Homeownership%:Homeowners Age 65 and Over', 'Homeownership%:Age 65 and Over Householders Homeowners', 'Homeownership:Renters Age 65 and Over', 
        'Homeownership%:Renters Age 65 and Over', 'Homeownership%:Age 65 and Over Householders Renters', 
        #poverty by age group
        'Poverty:Below Poverty Level 65 and Over', 'Poverty%:Below Poverty Level 65 and Over', 'Poverty%:65 and Over Below Poverty Level',
        #poverty total
        'Poverty:Population for Whom Poverty Status is Determined', 'Poverty: Population Below Poverty Level', 'Poverty%: Below Poverty Level',
        #living alone age group
        'LivingSituation:Living Alone 65 Over', 'LivingSituation%:65 Over that are Living Alone', 'LivingSituation:Living with Spouse 65 Over', 
        'LivingSituation%:65 Over that are Living with Spouse', 
        #grandparents as caregivers
        'Grandparents:Grandparents 60 Over, Primary Caregiver for Grandchild', 'Grandparents%:Grandparents 60 Over that are Primary Caregiver for Grandchild', 
        #lf stats
        'Employment:In Labor Force 65 Over', 'Employment%:Total Population 65 Over In Labor Force', 'Employment%:Population in LF 65 Over that are Unemployed', 
        #social security
        'SocialSecurity:Households with Social Security Income', 'SocialSecurity%:Households with Social Security Income'




        
            ]
data = data[cols]
data

,NAME,Age:65 and Older,Age%:65 and Older,HHIncome:Median for Householder Under 65 Over,Median Household Income,"HHIncome:65 Over Less than 10,000","HHIncome%:65 Over Less than 10,000","HHIncome:65 Over 10 to 14,999","HHIncome%:65 Over 10 to 14,999","HHIncome:65 Over 15 to 19,999","HHIncome%:65 Over 15 to 19,999","HHIncome:65 Over 20 to 24,999","HHIncome%:65 Over 20 to 24,999","HHIncome:65 Over 25 to 29,999","HHIncome%:65 Over 25 to 29,999","HHIncome:65 Over 30 to 34,999","HHIncome%:65 Over 30 to 34,999","HHIncome:65 Over 35 to 39,999","HHIncome%:65 Over 35 to 39,999","HHIncome:65 Over 40 to 44,999","HHIncome%:65 Over 40 to 44,999","HHIncome:65 Over 45 to 49,999","HHIncome%:65 Over 45 to 49,999","HHIncome:65 Over 50 to 59,999","HHIncome%:65 Over 50 to 59,999","HHIncome:65 Over 60 to 74,999","HHIncome%:65 Over 60 to 74,999","HHIncome:65 Over 75 to 99,999","HHIncome%:65 Over 75 to 99,999","HHIncome:65 Over 100 to 124,999","HHIncome%:65 Over 100 to 124,999","HHIncome:65 Over 125 to 149,999","HHIncome%:65 Over 125 to 149,999","HHIncome:65 Over 150 to 199,999","HHIncome%:65 Over 150 to 199,999",HHIncome:65 Over 200K or More,HHIncome%:65 Over 200K or More,CBAge:Cost Burdened Households Householder 65 Over,CBAge%:Households Cost Burdened Householder 65 Over,Homeownership:Householders Age 60 and Over,Homeownership:Homeowners Age 60 and Over,Homeownership%:Homeowners Age 60 and Over,Homeownership%:Age 60 and Over Householders Homeowners,Homeownership:Renters Age 60 and Over,Homeownership%:Renters Age 60 and Over,Homeownership%:Age 60 and Over Householders Renters,Homeownership:Householders Age 65 and Over,Homeownership:Homeowners Age 65 and Over,Homeownership%:Homeowners Age 65 and Over,Homeownership%:Age 65 and Over Householders Homeowners,Homeownership:Renters Age 65 and Over,Homeownership%:Renters Age 65 and Over,Homeownership%:Age 65 and Over Householders Renters,Poverty:Below Poverty Level 65 and Over,Poverty%:Below Poverty Level 65 and Over,Poverty%:65 and Over Below Poverty Level,Poverty:Population for Whom Poverty Status is Determined,Poverty: Population Below Poverty Level,Poverty%: Below Poverty Level,LivingSituation:Living Alone 65 Over,LivingSituation%:65 Over that are Living Alone,LivingSituation:Living with Spouse 65 Over,LivingSituation%:65 Over that are Living with Spouse,"Grandparents:Grandparents 60 Over, Primary Caregiver for Grandchild",Grandparents%:Grandparents 60 Over that are Primary Caregiver for Grandchild,Employment:In Labor Force 65 Over,Employment%:Total Population 65 Over In Labor Force,Employment%:Population in LF 65 Over that are Unemployed,SocialSecurity:Households with Social Security Income,SocialSecurity%:Households with Social Security Income
21,Tennessee,1187354.0,17.022464,44943.0,59695.0,44476.0,5.955966,56795.0,7.605655,57014.0,7.634982,47329.0,6.338023,46574.0,6.236918,42717.0,5.720411,39812.0,5.331391,39119.0,5.238588,37230.0,4.985624,61898.0,8.289019,68126.0,9.123036,78737.0,10.544,45250.0,6.059616,27095.0,3.628404,26197.0,3.508149,28378.0,3.800216,186394.0,24.960797,1021060.0,834643.0,44.656097,81.742797,186417.0,20.682,18.257203,746747.0,619957.0,33.169703,83.021023,126790.0,14.066693,16.978977,118416.0,12.766026,10.180131,6811613.0,927587.0,13.617729,317473.0,26.737856,312746.0,26.339744,20841.0,22.955171,208464.0,17.557022,3.789623,908626.0,32.797706


In [6]:
thelist = [data['HHIncome%:65 Over Less than 10,000'], data['HHIncome%:65 Over 10 to 14,999'], data['HHIncome%:65 Over 15 to 19,999'],
                data['HHIncome%:65 Over 20 to 24,999'], data['HHIncome%:65 Over 25 to 29,999'],
                data['HHIncome%:65 Over 30 to 34,999'], data['HHIncome%:65 Over 35 to 39,999'], data['HHIncome%:65 Over 40 to 44,999']]
data['HHIncome%:65 Over Making Less than 45,000'] = sum(thelist)

In [7]:
data

,NAME,Age:65 and Older,Age%:65 and Older,HHIncome:Median for Householder Under 65 Over,Median Household Income,"HHIncome:65 Over Less than 10,000","HHIncome%:65 Over Less than 10,000","HHIncome:65 Over 10 to 14,999","HHIncome%:65 Over 10 to 14,999","HHIncome:65 Over 15 to 19,999","HHIncome%:65 Over 15 to 19,999","HHIncome:65 Over 20 to 24,999","HHIncome%:65 Over 20 to 24,999","HHIncome:65 Over 25 to 29,999","HHIncome%:65 Over 25 to 29,999","HHIncome:65 Over 30 to 34,999","HHIncome%:65 Over 30 to 34,999","HHIncome:65 Over 35 to 39,999","HHIncome%:65 Over 35 to 39,999","HHIncome:65 Over 40 to 44,999","HHIncome%:65 Over 40 to 44,999","HHIncome:65 Over 45 to 49,999","HHIncome%:65 Over 45 to 49,999","HHIncome:65 Over 50 to 59,999","HHIncome%:65 Over 50 to 59,999","HHIncome:65 Over 60 to 74,999","HHIncome%:65 Over 60 to 74,999","HHIncome:65 Over 75 to 99,999","HHIncome%:65 Over 75 to 99,999","HHIncome:65 Over 100 to 124,999","HHIncome%:65 Over 100 to 124,999","HHIncome:65 Over 125 to 149,999","HHIncome%:65 Over 125 to 149,999","HHIncome:65 Over 150 to 199,999","HHIncome%:65 Over 150 to 199,999",HHIncome:65 Over 200K or More,HHIncome%:65 Over 200K or More,CBAge:Cost Burdened Households Householder 65 Over,CBAge%:Households Cost Burdened Householder 65 Over,Homeownership:Householders Age 60 and Over,Homeownership:Homeowners Age 60 and Over,Homeownership%:Homeowners Age 60 and Over,Homeownership%:Age 60 and Over Householders Homeowners,Homeownership:Renters Age 60 and Over,Homeownership%:Renters Age 60 and Over,Homeownership%:Age 60 and Over Householders Renters,Homeownership:Householders Age 65 and Over,Homeownership:Homeowners Age 65 and Over,Homeownership%:Homeowners Age 65 and Over,Homeownership%:Age 65 and Over Householders Homeowners,Homeownership:Renters Age 65 and Over,Homeownership%:Renters Age 65 and Over,Homeownership%:Age 65 and Over Householders Renters,Poverty:Below Poverty Level 65 and Over,Poverty%:Below Poverty Level 65 and Over,Poverty%:65 and Over Below Poverty Level,Poverty:Population for Whom Poverty Status is Determined,Poverty: Population Below Poverty Level,Poverty%: Below Poverty Level,LivingSituation:Living Alone 65 Over,LivingSituation%:65 Over that are Living Alone,LivingSituation:Living with Spouse 65 Over,LivingSituation%:65 Over that are Living with Spouse,"Grandparents:Grandparents 60 Over, Primary Caregiver for Grandchild",Grandparents%:Grandparents 60 Over that are Primary Caregiver for Grandchild,Employment:In Labor Force 65 Over,Employment%:Total Population 65 Over In Labor Force,Employment%:Population in LF 65 Over that are Unemployed,SocialSecurity:Households with Social Security Income,SocialSecurity%:Households with Social Security Income,"HHIncome%:65 Over Making Less than 45,000"
21,Tennessee,1187354.0,17.022464,44943.0,59695.0,44476.0,5.955966,56795.0,7.605655,57014.0,7.634982,47329.0,6.338023,46574.0,6.236918,42717.0,5.720411,39812.0,5.331391,39119.0,5.238588,37230.0,4.985624,61898.0,8.289019,68126.0,9.123036,78737.0,10.544,45250.0,6.059616,27095.0,3.628404,26197.0,3.508149,28378.0,3.800216,186394.0,24.960797,1021060.0,834643.0,44.656097,81.742797,186417.0,20.682,18.257203,746747.0,619957.0,33.169703,83.021023,126790.0,14.066693,16.978977,118416.0,12.766026,10.180131,6811613.0,927587.0,13.617729,317473.0,26.737856,312746.0,26.339744,20841.0,22.955171,208464.0,17.557022,3.789623,908626.0,32.797706,50.061935


In [8]:
data.to_csv('../data/agewell_20221Yr.csv')